In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
lstm = nn.LSTM(3, 3) #input dim 3, output dim 3
inputs = [torch.randn(1, 3) for _ in range(5)] #make a seq of 5
print('input1', inputs, '\n')

#init hidden state
hidden = (
    torch.randn(1, 1, 3),
    torch.randn(1, 1, 3)
)

inputs = torch.cat(inputs).view(len(inputs), 1, -1)
print('input2', inputs, '\n')
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))
print('hidden', hidden, '\n')
out, hidden = lstm(inputs, hidden)
print('out', out, '\n')
print('hidden', hidden, '\n')

input1 [tensor([[-0.5525,  0.6355, -0.3968]]), tensor([[-0.6571, -1.6428,  0.9803]]), tensor([[-0.0421, -0.8206,  0.3133]]), tensor([[-1.1352,  0.3773, -0.2824]]), tensor([[-2.5667, -1.4303,  0.5009]])] 

input2 tensor([[[-0.5525,  0.6355, -0.3968]],

        [[-0.6571, -1.6428,  0.9803]],

        [[-0.0421, -0.8206,  0.3133]],

        [[-1.1352,  0.3773, -0.2824]],

        [[-2.5667, -1.4303,  0.5009]]]) 

hidden (tensor([[[-0.1473,  0.6272,  1.0935]]]), tensor([[[ 0.0939,  1.2381, -1.3459]]])) 

out tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>) 

hidden (tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward>)) 



In [3]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [16]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"]),
    (["book"], ["NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [5]:
inputs = prepare_sequence(training_data[2][0], word_to_ix)
targets = prepare_sequence(training_data[2][1], tag_to_ix)
print(inputs, targets)

tensor([8]) tensor([1])


In [17]:
class LSTMTagger(nn.Module):
    
    def __init__(
        self,
        embedding_dim,
        hidden_dim,
        vocab_size,
        target_size
    ):
        super(LSTMTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, target_size)
        self.sig = nn.Sigmoid()
    
    def forward(self, sentence):
        print('sentence (per word)', sentence, '\n')
        embeds = self.word_embeddings(sentence)
        print('embed', embeds, '\n')
        print('embed view', embeds.view(len(sentence), 1, -1), '\n')
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        print('lstm out', lstm_out, '\n')
        print('lstm out sq', lstm_out.squeeze(), '\n')
        print('lstm out view', lstm_out.view(len(sentence), -1), '\n')
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        print('tag_space', tag_space, '\n')
#         tag_scores = F.log_softmax(tag_space, dim=1)
        tag_scores = self.sig(tag_space)
        print('tag_scores', tag_scores, '\n')
        return tag_scores

In [18]:
model = LSTMTagger(
    EMBEDDING_DIM,
    HIDDEN_DIM,
    len(word_to_ix),
    len(tag_to_ix)
)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [20]:
print(prepare_sequence(training_data[0][0], word_to_ix))

tensor([0, 1, 2, 3, 4])


In [21]:
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

sentence (per word) tensor([0, 1, 2, 3, 4]) 

embed tensor([[-1.0301,  0.6039,  0.9831,  3.2373, -0.0390, -0.4583],
        [-0.1672,  0.5562, -0.6607, -2.7947,  0.4582,  0.0534],
        [-0.6820,  1.6040, -0.4936,  1.8431,  0.8171, -0.6775],
        [ 1.7857, -1.5699, -1.0271,  0.1954,  0.9382,  1.5122],
        [ 0.4242,  0.9676,  0.2028, -0.7651,  0.4003, -0.9219]]) 

embed view tensor([[[-1.0301,  0.6039,  0.9831,  3.2373, -0.0390, -0.4583]],

        [[-0.1672,  0.5562, -0.6607, -2.7947,  0.4582,  0.0534]],

        [[-0.6820,  1.6040, -0.4936,  1.8431,  0.8171, -0.6775]],

        [[ 1.7857, -1.5699, -1.0271,  0.1954,  0.9382,  1.5122]],

        [[ 0.4242,  0.9676,  0.2028, -0.7651,  0.4003, -0.9219]]]) 

lstm out tensor([[[ 2.7095e-01, -4.4941e-03,  3.6637e-01,  1.3571e-01, -8.9916e-02,
          -5.2677e-02]],

        [[ 6.5759e-02,  9.2645e-02, -1.6088e-02, -2.2010e-01,  1.4048e-01,
           1.4815e-01]],

        [[ 3.5869e-01,  6.9950e-02,  2.3049e-01, -1.2563e-01, -5.4

In [22]:
for epoch in range(1):
    for sentence, tags in training_data:
        model.zero_grad()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        tag_scores = model(sentence_in)
        print('targets', targets)
        loss = loss_function(tag_scores, targets)
        print('loss', loss)
        
        loss.backward()
        optimizer.step()
        break
    break

sentence (per word) tensor([0, 1, 2, 3, 4]) 

embed tensor([[-1.0301,  0.6039,  0.9831,  3.2373, -0.0390, -0.4583],
        [-0.1672,  0.5562, -0.6607, -2.7947,  0.4582,  0.0534],
        [-0.6820,  1.6040, -0.4936,  1.8431,  0.8171, -0.6775],
        [ 1.7857, -1.5699, -1.0271,  0.1954,  0.9382,  1.5122],
        [ 0.4242,  0.9676,  0.2028, -0.7651,  0.4003, -0.9219]],
       grad_fn=<EmbeddingBackward>) 

embed view tensor([[[-1.0301,  0.6039,  0.9831,  3.2373, -0.0390, -0.4583]],

        [[-0.1672,  0.5562, -0.6607, -2.7947,  0.4582,  0.0534]],

        [[-0.6820,  1.6040, -0.4936,  1.8431,  0.8171, -0.6775]],

        [[ 1.7857, -1.5699, -1.0271,  0.1954,  0.9382,  1.5122]],

        [[ 0.4242,  0.9676,  0.2028, -0.7651,  0.4003, -0.9219]]],
       grad_fn=<ViewBackward>) 

lstm out tensor([[[ 2.7095e-01, -4.4941e-03,  3.6637e-01,  1.3571e-01, -8.9916e-02,
          -5.2677e-02]],

        [[ 6.5759e-02,  9.2645e-02, -1.6088e-02, -2.2010e-01,  1.4048e-01,
           1.4815e-01]],


In [55]:
with torch.no_grad():
    inputs = prepare_sequence(training_data[2][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

sentence (per word) tensor([8]) 

embed tensor([[ 1.2678, -1.3178,  2.1215, -2.3513, -0.3042, -1.2749]]) 

embed view tensor([[[ 1.2678, -1.3178,  2.1215, -2.3513, -0.3042, -1.2749]]]) 

lstm out tensor([[[-0.0943,  0.1698,  0.0422, -0.1209, -0.0135, -0.3966]]]) 

lstm out view tensor([[-0.0943,  0.1698,  0.0422, -0.1209, -0.0135, -0.3966]]) 

tag_space tensor([[-0.0651,  0.0510,  0.4032]]) 

tag_scores tensor([[0.4837, 0.5128, 0.5995]]) 

tensor([[0.4837, 0.5128, 0.5995]])
